<a href="https://colab.research.google.com/github/Scarlet-Thunder/Amazon-Clone-CSS-HTML-/blob/main/Movie_Recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installing kaggle module
!pip install kaggle

In [ ]:
# getting authentication key
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gautamahuja","key":"ee183fb5d65529ef9c39cad4565c8f94"}'}

In [ ]:
# moving kaggle.json into a hidden directory, and changing permissions for it
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# downloading movie dataset
!kaggle datasets download -d tmdb/tmdb-movie-metadata

 56% 5.00M/8.89M [00:00<00:00, 46.5MB/s]
100% 8.89M/8.89M [00:00<00:00, 71.7MB/s]


In [ ]:
# listing files in current directory
!ls

kaggle.json  sample_data  tmdb-movie-metadata.zip


In [ ]:
# unzipping files
!unzip tmdb-movie-metadata.zip

Archive:  tmdb-movie-metadata.zip
  inflating: tmdb_5000_credits.csv   
  inflating: tmdb_5000_movies.csv    


In [ ]:
# creating dataframe
import pandas as pd
import numpy as np
movies_df = pd.read_csv('/content/tmdb_5000_movies.csv')
credits_df = pd.read_csv('/content/tmdb_5000_credits.csv')

In [ ]:
# info of credits dataframe
credits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [ ]:
# changing col name to merge dataframes
credits_df.rename(columns = {'movie_id' : 'id'} , inplace = True)
credits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      4803 non-null   int64 
 1   title   4803 non-null   object
 2   cast    4803 non-null   object
 3   crew    4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [ ]:
# merging dataframes
common_df = pd.merge(movies_df , credits_df , on = 'id')
common_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [ ]:
# dropping columns : cleaning dataset
common_df.drop(['homepage', 'title_x', 'title_y', 'production_companies'] , axis = 1 , inplace = True)
common_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   id                    4803 non-null   int64  
 3   keywords              4803 non-null   object 
 4   original_language     4803 non-null   object 
 5   original_title        4803 non-null   object 
 6   overview              4800 non-null   object 
 7   popularity            4803 non-null   float64
 8   production_countries  4803 non-null   object 
 9   release_date          4802 non-null   object 
 10  revenue               4803 non-null   int64  
 11  runtime               4801 non-null   float64
 12  spoken_languages      4803 non-null   object 
 13  status                4803 non-null   object 
 14  tagline               3959 non-null   object 
 15  vote_average         

In [ ]:
common_df.dropna(inplace = True)
common_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3959 entries, 0 to 4801
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                3959 non-null   int64  
 1   genres                3959 non-null   object 
 2   id                    3959 non-null   int64  
 3   keywords              3959 non-null   object 
 4   original_language     3959 non-null   object 
 5   original_title        3959 non-null   object 
 6   overview              3959 non-null   object 
 7   popularity            3959 non-null   float64
 8   production_countries  3959 non-null   object 
 9   release_date          3959 non-null   object 
 10  revenue               3959 non-null   int64  
 11  runtime               3959 non-null   float64
 12  spoken_languages      3959 non-null   object 
 13  status                3959 non-null   object 
 14  tagline               3959 non-null   object 
 15  vote_average         

In [ ]:
# calculating weighted_average
R = common_df['vote_average']
v = common_df['vote_count']
C = common_df['vote_average'].mean()
m = common_df['vote_count'].quantile(0.9)
print(C , m)

# adding weighted_average col
common_df['weighted_rating'] = (R*v + C*m) / (v + m)

6.1819398838090445 2104.2000000000003


In [ ]:
# sorting dataframe
common_df.sort_values('weighted_rating' , ascending = False , inplace = True)

In [ ]:
# looking at columns : cast, crew, keywords, original title, genres
common_df[['original_title' , 'cast' , 'crew' , 'keywords' , 'genres']].head(3)

,original_title,cast,crew,keywords,genres
1881,The Shawshank Redemption,"[{""cast_id"": 3, ""character"": ""Andy Dufresne"", ...","[{""credit_id"": ""52fe4231c3a36847f800b153"", ""de...","[{""id"": 378, ""name"": ""prison""}, {""id"": 417, ""n...","[{""id"": 18, ""name"": ""Drama""}, {""id"": 80, ""name..."
662,Fight Club,"[{""cast_id"": 4, ""character"": ""The Narrator"", ""...","[{""credit_id"": ""55731b8192514111610027d7"", ""de...","[{""id"": 825, ""name"": ""support group""}, {""id"": ...","[{""id"": 18, ""name"": ""Drama""}]"
65,The Dark Knight,"[{""cast_id"": 35, ""character"": ""Bruce Wayne"", ""...","[{""credit_id"": ""55a0eb4a925141296b0010f8"", ""de...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""id"": 18, ""name"": ""Drama""}, {""id"": 28, ""name..."


In [ ]:
# check the datatype of elements
type(common_df.loc[0].at['cast'])

str

In [ ]:
# changing datatype of list
from ast import literal_eval

features = ['cast' , 'crew', 'keywords', 'genres']
for feature in features:
  common_df[feature] = common_df[feature].apply(literal_eval)

In [ ]:
# testing the datatype again
type(common_df.loc[0].at['cast'])

list

In [ ]:
# printing one element from crew column

In [ ]:
# adding director column in our dataframe

In [ ]:
# print the title and director column

In [ ]:
# printing first element in cast column

In [ ]:
# printing first element in keywords column

In [ ]:
# printing first element in genres column

In [ ]:
# extracting names

In [ ]:
# verify the changes

In [ ]:
# cleaning data

In [ ]:
# verify the changes

In [ ]:
# try the .join() method

In [ ]:
# creating soup : merging all the data

In [ ]:
# verifying the output

In [ ]:
# creating vectors or matrix

In [ ]:
# create the cosine similarity classifier

In [ ]:
# resetting the index

In [ ]:
# method to find similar movies

In [ ]:
# converting the df into csv

In [ ]:
# downloading the csv file